In [1]:
#Importing the necessary libraries for the script
import geemap
import ee
from pathlib import Path
import geopandas as gpd
from functions import *

In [14]:
#Authenticating the Google Earth Engine API
ee.Authenticate()
ee.Initialize()
geemap.ee_initialize()

In [15]:
#Setting the Out Directory if not exists creating it
out_dir = Path('../Data')
out_dir.mkdir(exist_ok=True)

#Shapefile directory of the area of interest
shp_dir = Path('../Shapefile/BegnasArea.shp')

In [16]:
#Define Seasons
# Define seasons and their corresponding timeframes for multiple years
seasons = {
    'Spring': [('2022-03-01', '2022-05-31'), ('2023-03-01', '2023-05-31')],
    'Summer': [('2022-06-01', '2022-08-31'), ('2023-06-01', '2023-08-31')],
    'Fall': [('2022-09-01', '2022-11-30'), ('2023-09-01', '2023-11-30')],
    'Winter': [('2021-12-01', '2022-02-28'), ('2022-12-01', '2023-02-28'), ('2023-12-01', '2024-02-28')]
}

In [25]:
studyarea = get_eesupported_roi(shp_dir)

# Earth Engine Collection
dataset_landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filter(ee.Filter.lt('CLOUD_COVER', 5))
dataset_landsat9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filter(ee.Filter.lt('CLOUD_COVER', 5))


In [26]:
# Function to apply scaling factors
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

In [27]:
# Process and save data for each season across multiple years
for season, timeframes in seasons.items():
    # Combine datasets for the specific season across multiple years
    dataset_landsat8_season = ee.ImageCollection([])
    dataset_landsat9_season = ee.ImageCollection([])

    for timeframe in timeframes:
        dataset_landsat8_season = dataset_landsat8_season.merge(dataset_landsat8.filterDate(timeframe[0], timeframe[1]).map(apply_scale_factors))
        dataset_landsat9_season = dataset_landsat9_season.merge(dataset_landsat9.filterDate(timeframe[0], timeframe[1]).map(apply_scale_factors))

    # Get median composite image across all years and clip to study area
    landsat_8_season = dataset_landsat8_season.median().clip(studyarea)
    landsat_9_season = dataset_landsat9_season.median().clip(studyarea)

    # Create season-wise directory
    season_dir = out_dir / season
    os.makedirs(season_dir, exist_ok=True)

    # Save data into season folder
    save_data(season_dir, landsat_8_season, f'landsat_8{season}.tif', studyarea)
    save_data(season_dir, landsat_9_season, f'landsat_9{season}.tif', studyarea)


landsat_8Summer.tif: |          | 0.00/13.6M (raw) [  0.0%] in 00:00 (eta:     ?)

landsat_9Summer.tif: |          | 0.00/13.6M (raw) [  0.0%] in 00:00 (eta:     ?)

In [22]:
#Save the LULC data
LULC = get_lulc(studyarea)
geemap.download_ee_image(LULC, '../Data/LULC5m.tif', scale= 5, region=studyarea, crs='EPSG:4326')



LULC5m.tif: |          | 0.00/3.20M (raw) [  0.0%] in 00:00 (eta:     ?)